# Task 4.1 Demonstrate execution Modes

## Objective 1 Execute on hardware

run your circuits on hardware and produce the outputs of the quantum computation

Guide:
* Run with Primitives
* Configure runtime options
* execution modes
* manage jobs
* qpu and platform information

## Objective 2 execution modes

* Job mode : single request of sampler or estimator, specify mode=backend when you initialize the primitive
* Batch mode: multi-job workload made up of independently executable jobs, specify mode=batch
* Session mode: dedicated window to run a multi-job workload , used for variational algorithms, specify mode=session

## Objective 3: Choose execution mode

* Batch
    * the entire batch is scheduled together
    * classicasl computation is run in parallel , signficantly faster than running them serially
    * minimal delay between jobs
    * more flexible to work with , you can cancel remaining jobs if result doesn't meet your expectations
    * less expensive than sessions
* Session
    * all functionality from batch mode
    * dedicated access to qpu
    * useful for workloads than don't have all inputs
* Job
    * easiest when running a small experiment
    * might run sooner than batch

## Objective 4 : Run Jobs in Batch

In [7]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)
 
service = QiskitRuntimeService()

In [2]:
backend = service.least_busy(operational=True, simulator=False)
batch = Batch(backend=backend)
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
# Close the batch because no context manager was used.
batch.close()

In [12]:
from qiskit_ibm_runtime import (
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)
 
backend = service.least_busy(operational=True, simulator=False)
with Batch(backend=backend):
    estimator = Estimator()
    sampler = Sampler()

Batch Length is 10 minutes for open plan , 8 hours for paid plans

In [ ]:
with Batch(backend=backend) as batch:
    estimator = Estimator()
    sampler = Sampler()
    job1 = estimator.run([estimator_pub])
    job2 = sampler.run([sampler_pub])
 
# The batch is no longer accepting jobs but the submitted job will run to completion.
result = job1.result()
result2 = job2.result()

[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000016FC6E69C50>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000016FC6E69C50>]


TypeError: Invalid observable type: <class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>

In [ ]:
batch = Batch(backend=backend)
 
# If using qiskit-ibm-runtime earlier than 0.24.0, change `mode=` to `batch=`
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
job1 = estimator.run([estimator_pub])
job2 = sampler.run([sampler_pub])
print(f"Result1: {job1.result()}")
print(f"Result2: {job2.result()}")
 
# Manually close the batch. Running and queued jobs will run to completion.
batch.close()

In [16]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    batch,
    SamplerV2 as Sampler,
)
 
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
 
with Batch(backend=backend) as batch:
    print(batch.details())

{'id': '5773078b-dd45-40a2-9dd2-9ee1a50a8fd4', 'backend_name': 'ibm_torino', 'interactive_timeout': 1, 'max_time': 600, 'active_timeout': 600, 'state': 'open', 'accepting_jobs': True, 'last_job_started': None, 'last_job_completed': None, 'started_at': None, 'closed_at': None, 'activated_at': None, 'mode': 'batch', 'usage_time': None}


In [18]:
from qiskit_ibm_runtime import SamplerV2 as Sampler, Batch
from qiskit.circuit.random import random_circuit
 
max_circuits = 100
circuits = [random_circuit(5, 5) for _ in range(5 * max_circuits)]
all_partitioned_circuits = []
for i in range(0, len(circuits), max_circuits):
    all_partitioned_circuits.append(circuits[i : i + max_circuits])
jobs = []
start_idx = 0
 
with Batch(backend=backend):
    sampler = Sampler()
    for partitioned_circuits in all_partitioned_circuits:
        job = sampler.run(partitioned_circuits)
        jobs.append(job)

IBMInputValueError: 'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle of 4.034560455146796 has been provided.'